<a href="https://colab.research.google.com/github/mateor32/modelos1-20252-competenciaKaggle/blob/main/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
print("¡Drive conectado!")
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import warnings
import os



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
¡Drive conectado!


In [ ]:
warnings.filterwarnings('ignore')


In [ ]:
# ----------------------------------------------------------------------
# 0. CONFIGURACIÓN Y PARÁMETROS OPTIMIZADOS
# ----------------------------------------------------------------------

# AJUSTA ESTA RUTA SI ES NECESARIO
BASE_PATH = "/content/drive/MyDrive/Competencia_Saber_Pro_Final/"

In [ ]:
# Parámetros optimizados (Tuning Suave)
LGBM_TUNED_PARAMS = {
    'objective': 'multiclass',
    'num_class': 4,
    'n_estimators': 1000,          # Más iteraciones
    'learning_rate': 0.03,         # Menos agresivo para mayor precisión
    'num_leaves': 35,
    'reg_lambda': 0.1,             # Regularización suave
    'colsample_bytree': 0.8,       # Regularización de columnas
    'random_state': 42,
    'verbose': -1,
    'n_jobs': -1
}

print("Iniciando Solución Final 99 con Tuning y Feature Engineering...")



Iniciando Solución Final 99 con Tuning y Feature Engineering...


In [ ]:
# ----------------------------------------------------------------------
# 1. CARGA DE DATOS Y LIMPIEZA INICIAL (CON FIX ROBUSTO)
# ----------------------------------------------------------------------

# Definición de la función de limpieza antes de usarla
def clean_cols(df):
    """Limpia nombres de columnas (Consistente con 02 - prepocesado.ipynb)"""
    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.replace('.', '_', regex=False)
    df.columns = df.columns.str.upper()
    return df


In [ ]:
print("1. Cargando datos...")
df_train = None
df_test = None


1. Cargando datos...


In [ ]:
try:
    df_train = pd.read_csv(f"{BASE_PATH}train.csv")
    df_test = pd.read_csv(f"{BASE_PATH}test.csv")
    print(f"Éxito al cargar {len(df_train)} filas de datos.")
except FileNotFoundError:
    print(f"ERROR CRÍTICO: No se encontraron los archivos en la ruta: {BASE_PATH}")
    print("Por favor, verifica que Google Drive esté montado y la ruta BASE_PATH sea correcta.")
    # Detenemos la ejecución si la carga falla
    raise SystemExit("Deteniendo la ejecución. Archivos de datos no encontrados.")


df_train = clean_cols(df_train)
df_test = clean_cols(df_test)

TARGET_COL = 'RENDIMIENTO_GLOBAL'
y = df_train[TARGET_COL]
X = df_train.drop(TARGET_COL, axis=1)
test_ids = df_test['ID']
X_test = df_test.copy()



✅ Éxito al cargar 692500 filas de datos.


In [ ]:
# Mapeo de Etiquetas (numéricas para el modelo)
unique_labels = sorted(y.unique())
label_map = {label: i for i, label in enumerate(unique_labels)}
reverse_label_map = {i: label for label, i in label_map.items()}
y_encoded = y.map(label_map)


In [ ]:
# ----------------------------------------------------------------------
# 2. PREPROCESAMIENTO Y FEATURE ENGINEERING
# ----------------------------------------------------------------------

print("2. Aplicando Feature Engineering (INDICADOR_PROMEDIO) y Preprocesamiento OHE...")


2. Aplicando Feature Engineering (INDICADOR_PROMEDIO) y Preprocesamiento OHE...


In [ ]:
# Identificar columnas de indicadores
indicador_cols = [col for col in X.columns if 'INDICADOR_' in col]


In [ ]:
# 🚨 NUEVA CARACTERÍSTICA: INDICADOR PROMEDIO
X['INDICADOR_PROMEDIO'] = X[indicador_cols].mean(axis=1)
X_test['INDICADOR_PROMEDIO'] = X_test[indicador_cols].mean(axis=1)




In [ ]:
# Identificar columnas
num_cols = [col for col in X.select_dtypes(include=['int64', 'float64']).columns.tolist() if col != 'ID']
cat_cols = X.select_dtypes(include=['object']).columns.tolist()


In [ ]:
# Imputación (Moda y Media)
imputer_cte = SimpleImputer(strategy='most_frequent')
X[cat_cols] = imputer_cte.fit_transform(X[cat_cols])
X_test[cat_cols] = imputer_cte.transform(X_test[cat_cols])

imputer_num = SimpleImputer(strategy='mean')
X[num_cols] = imputer_num.fit_transform(X[num_cols])
X_test[num_cols] = imputer_num.transform(X_test[num_cols])



In [ ]:
# One-Hot Encoding (OHE)
combined_encoded = pd.concat([X.drop('ID', axis=1), X_test.drop('ID', axis=1)], keys=['train', 'test'])
combined_encoded = pd.get_dummies(combined_encoded.drop(TARGET_COL, axis=1, errors='ignore'), drop_first=True, dtype=bool)

X_train_features = combined_encoded.loc['train'].reset_index(drop=True)
X_test_features = combined_encoded.loc['test'].reset_index(drop=True)


In [ ]:
# 🛠 FIX: Sanitización y Unicidad de Nombres (Obligatorio para LightGBM)
print("Sanitizando nombres de columnas...")
def sanitize_and_uniquify_columns(df):
    """Limpia caracteres especiales y asegura unicidad de nombres."""
    df.columns = df.columns.str.replace(r'[<>()\[\]{},.:;\'\"-/]', '_', regex=True)
    df.columns = df.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    df.columns = [''.join(c if c.isalnum() else '_' for c in col) for col in df.columns]

    cols = df.columns.tolist()
    seen = {}
    new_cols = []
    for item in cols:
        counter = seen.get(item, 0)
        if counter > 0:
            new_cols.append(f'{item}_{counter}')
        else:
            new_cols.append(item)
        seen[item] = counter + 1

    df.columns = new_cols
    return df

X_train_features = sanitize_and_uniquify_columns(X_train_features)
X_test_features = sanitize_and_uniquify_columns(X_test_features)


Sanitizando nombres de columnas...


In [ ]:
# Normalización MinMax
print("Aplicando MinMax Scaling...")
all_normalized_cols = [col for col in X_train_features.columns if 'INDICADOR_' in col or 'PERIODO_ACADEMICO' in col]

scaler = MinMaxScaler()
X_train_features[all_normalized_cols] = scaler.fit_transform(X_train_features[all_normalized_cols])
X_test_features[all_normalized_cols] = scaler.transform(X_test_features[all_normalized_cols])


Aplicando MinMax Scaling...


In [ ]:
# ----------------------------------------------------------------------
# 3. MODELADO: LIGHTGBM (TUNING SUAVE)
# ----------------------------------------------------------------------

print("3. Entrenando LightGBM con tuning suave (n=1000, lr=0.03)...")

# Crear un set de validación (10% para Early Stopping)
X_t, X_val, y_t, y_val = train_test_split(
    X_train_features,
    y_encoded,
    test_size=0.1,
    random_state=42,
    stratify=y_encoded
)

model = lgb.LGBMClassifier(**LGBM_TUNED_PARAMS)


3. Entrenando LightGBM con tuning suave (n=1000, lr=0.03)...


In [ ]:
# Entrenamiento con Early Stopping
model.fit(
    X_t, y_t,
    eval_set=[(X_val, y_val)],
    eval_metric='multi_logloss',
    callbacks=[lgb.early_stopping(100, verbose=False)]
)



,boosting_type,'gbdt'
,num_leaves,35
,max_depth,-1
,learning_rate,0.03
,n_estimators,1000
,subsample_for_bin,200000
,objective,'multiclass'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [ ]:
# Predicción
predictions_numeric = model.predict(X_test_features)


In [ ]:
# Mapear de vuelta a las etiquetas de texto
final_predictions = pd.Series(predictions_numeric).map(reverse_label_map)


In [ ]:
# Generar el archivo de sumisión
submission_df = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': final_predictions
})

submission_filename = 'submission_99_TUNED_V5.csv'
submission_df.to_csv(submission_filename, index=False)

print(f"\n=========================================================")
print(f"🎉 ÉXITO: Solución mejorada con tuning y nueva feature. Archivo generado: {submission_filename}")
print("¡Sube este archivo a Kaggle para ver la mejora de tu score!")
print("=========================================================")




🎉 ÉXITO: Solución mejorada con tuning y nueva feature. Archivo generado: submission_99_TUNED_V5.csv
¡Sube este archivo a Kaggle para ver la mejora de tu score!
